# Piecewise Efficiency: Heat Pump with Variable COP

## User Story

> *You're installing a heat pump for a commercial building. The heat pump's efficiency (COP - Coefficient of Performance) varies with the outdoor temperature: it's more efficient in mild weather than in cold weather. You want to model this realistically to accurately predict operating costs.*

This notebook introduces:

- **Piecewise linear functions**: Approximate non-linear behavior
- **Variable efficiency**: COP changes with operating conditions
- **LinearConverter with segments**: Multiple operating points
- **Piecewise effects**: Non-linear cost curves

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

## The Problem: Variable Heat Pump Efficiency

A heat pump's COP (Coefficient of Performance) depends on the temperature difference between source and sink:

- **Mild weather** (10°C outside): COP ≈ 4.5 (1 kWh electricity → 4.5 kWh heat)
- **Cold weather** (-5°C outside): COP ≈ 2.5 (1 kWh electricity → 2.5 kWh heat)

This non-linear relationship can be approximated using piecewise linear segments.

## Define Time Series Data

In [ ]:
# One winter week
timesteps = pd.date_range('2024-01-22', periods=168, freq='h')
hours = np.arange(168)
hour_of_day = hours % 24

# Outdoor temperature: daily cycle with cold nights
temp_base = 2  # Average temp
temp_amplitude = 5  # Daily variation
outdoor_temp = temp_base + temp_amplitude * np.sin((hour_of_day - 6) * np.pi / 12)

# Add day-to-day variation
np.random.seed(789)
daily_offset = np.repeat(np.random.uniform(-3, 3, 7), 24)
outdoor_temp = outdoor_temp + daily_offset

In [ ]:
# Heat demand: inversely related to outdoor temp (higher demand when colder)
heat_demand = 200 - 8 * outdoor_temp
heat_demand = np.clip(heat_demand, 100, 300)

In [ ]:
# Visualize with plotly - using xarray and faceting
profiles = xr.Dataset(
    {
        'Outdoor Temp [°C]': xr.DataArray(outdoor_temp, dims=['time'], coords={'time': timesteps}),
        'Heat Demand [kW]': xr.DataArray(heat_demand, dims=['time'], coords={'time': timesteps}),
    }
)

df = profiles.to_dataframe().reset_index().melt(id_vars='time', var_name='variable', value_name='value')
fig = px.line(df, x='time', y='value', facet_col='variable', height=300)
fig.update_yaxes(matches=None, showticklabels=True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
fig

## Calculate Time-Varying COP

The COP depends on outdoor temperature. We use a simplified Carnot-based formula:

In [ ]:
# COP calculation (simplified): Real COP ≈ 0.45 * Carnot COP
T_supply = 45 + 273.15  # Supply temperature 45°C in Kelvin
T_source = outdoor_temp + 273.15  # Outdoor temp in Kelvin

carnot_cop = T_supply / (T_supply - T_source)
real_cop = 0.45 * carnot_cop
real_cop = np.clip(real_cop, 2.0, 5.0)  # Physical limits

In [ ]:
# Plot COP vs temperature using plotly
px.scatter(
    x=outdoor_temp,
    y=real_cop,
    title='Heat Pump COP vs Outdoor Temperature',
    labels={'x': 'Outdoor Temperature [°C]', 'y': 'COP'},
    opacity=0.5,
)

## Approach 1: Simple Model with Time-Varying Efficiency

The simplest approach: use time-varying conversion factors directly.

In [ ]:
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    fx.Bus('Electricity', carrier='electricity'),
    fx.Bus('Heat', carrier='heat'),
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    fx.Source('Grid', outputs=[fx.Flow('Elec', bus='Electricity', size=500, effects_per_flow_hour=0.30)]),
    fx.LinearConverter(
        'HeatPump',
        inputs=[fx.Flow('Elec', bus='Electricity', size=150)],
        outputs=[fx.Flow('Heat', bus='Heat', size=500)],
        conversion_factors=[{'Elec': real_cop, 'Heat': 1}],  # Time-varying COP
    ),
    fx.Sink('Building', inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand)]),
)

flow_system.optimize(fx.solvers.HighsSolver())

In [ ]:
flow_system.statistics.plot.balance('Heat')

In [ ]:
flow_system.statistics.plot.balance('Electricity')

## Approach 2: Simple vs Piecewise Efficiency - Model Refinement

Real equipment often has non-linear efficiency curves. Let's compare:
1. **Simple model**: Constant average efficiency
2. **Refined model**: Piecewise linear efficiency that varies with load

This demonstrates how to progressively refine a model for more accurate results.

In [ ]:
# Gas engine efficiency varies with load:
# - Part load: lower efficiency
# - Mid load: optimal efficiency (~42%)
# - Full load: slightly lower efficiency

SIMPLE_EFFICIENCY = 0.38  # Average efficiency for simple model

# Piecewise model: efficiency varies by operating segment
piecewise_efficiency = fx.PiecewiseConversion(
    {
        'Fuel': fx.Piecewise(
            [
                fx.Piece(start=78, end=132),  # Part load
                fx.Piece(start=132, end=179),  # Mid load (optimal)
                fx.Piece(start=179, end=250),  # High load
            ]
        ),
        'Elec': fx.Piecewise(
            [
                fx.Piece(start=25, end=50),
                fx.Piece(start=50, end=75),
                fx.Piece(start=75, end=100),
            ]
        ),
    }
)

In [ ]:
# 2-day demand profile
timesteps_simple = pd.date_range('2024-01-22', periods=48, freq='h')
elec_demand_simple = np.concatenate([np.linspace(30, 90, 24), np.linspace(90, 30, 24)])

# MODEL 1: Simple constant efficiency
fs_simple = fx.FlowSystem(timesteps_simple)
fs_simple.add_elements(
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas', size=300)],
        outputs=[fx.Flow('Elec', bus='Electricity', size=100)],
        conversion_factors=[{'Fuel': 1, 'Elec': SIMPLE_EFFICIENCY}],
    ),
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand_simple)]),
)
fs_simple.optimize(fx.solvers.HighsSolver())

In [ ]:
# MODEL 2: Piecewise efficiency (load-dependent)
fs_piecewise = fx.FlowSystem(timesteps_simple)
fs_piecewise.add_elements(
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas')],
        outputs=[fx.Flow('Elec', bus='Electricity')],
        piecewise_conversion=piecewise_efficiency,
    ),
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand_simple)]),
)
fs_piecewise.optimize(fx.solvers.HighsSolver())

In [ ]:
# Visualize the piecewise conversion curve
fs_piecewise.components['GasEngine'].piecewise_conversion.plot(
    x_flow='Fuel', title='Gas Engine: Fuel Input vs Electricity Output'
)

In [ ]:
# Compare: Simple vs Piecewise efficiency
print(f'Simple model:    {fs_simple.solution["costs"].item():.2f} €')
print(f'Piecewise model: {fs_piecewise.solution["costs"].item():.2f} €')

## Approach 3: Simple vs Piecewise Investment Costs

Investment costs often have economies of scale - larger systems cost less per unit.

In [ ]:
fs_piecewise.statistics.plot.balance('Gas')

In [ ]:
# Investment costs (daily amortized)
SIMPLE_INVEST_COST = 0.20  # €/kWh constant

# Piecewise: economies of scale (0.20 → 0.15 → 0.10 €/kWh)
piecewise_invest_costs = fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise(
        [
            fx.Piece(start=0, end=100),
            fx.Piece(start=100, end=300),
            fx.Piece(start=300, end=600),
        ]
    ),
    piecewise_shares={
        'costs': fx.Piecewise(
            [
                fx.Piece(start=0, end=20),  # 0.20 €/kWh
                fx.Piece(start=20, end=50),  # 0.15 €/kWh
                fx.Piece(start=50, end=80),  # 0.10 €/kWh
            ]
        )
    },
)

In [ ]:
# Base system: Price arbitrage scenario (3 days)
timesteps_invest = pd.date_range('2024-01-22', periods=72, freq='h')

heat_demand_invest = np.tile(
    np.concatenate(
        [
            np.full(8, 100),
            np.full(4, 120),
            np.full(4, 110),
            np.full(4, 130),
            np.full(4, 105),
        ]
    ),
    3,
)

energy_price_invest = np.tile(
    np.concatenate(
        [
            np.full(8, 0.08),
            np.full(4, 0.20),
            np.full(4, 0.12),
            np.full(4, 0.25),
            np.full(4, 0.10),
        ]
    ),
    3,
)

fs_base = fx.FlowSystem(timesteps_invest)
fs_base.add_elements(
    fx.Bus('Heat', carrier='heat'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('HeatSource', outputs=[fx.Flow('Heat', bus='Heat', size=300, effects_per_flow_hour=energy_price_invest)]),
    fx.Sink('HeatSink', inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand_invest)]),
)

In [ ]:
# MODEL 1: Simple linear investment costs
fs_simple_invest = fs_base.copy()
fs_simple_invest.add_elements(
    fx.Storage(
        'ThermalStorage',
        charging=fx.Flow('charge', bus='Heat', size=200),
        discharging=fx.Flow('discharge', bus='Heat', size=200),
        capacity_in_flow_hours=fx.InvestParameters(
            effects_of_investment_per_size=SIMPLE_INVEST_COST,
            minimum_size=0,
            maximum_size=600,
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.005,
    ),
)
fs_simple_invest.optimize(fx.solvers.HighsSolver())

In [ ]:
# MODEL 2: Piecewise investment costs (economies of scale)
fs_piecewise_invest = fs_base.copy()
fs_piecewise_invest.add_elements(
    fx.Storage(
        'ThermalStorage',
        charging=fx.Flow('charge', bus='Heat', size=200),
        discharging=fx.Flow('discharge', bus='Heat', size=200),
        capacity_in_flow_hours=fx.InvestParameters(
            piecewise_effects_of_investment=piecewise_invest_costs,
            minimum_size=0,
            maximum_size=600,
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.005,
    ),
)
fs_piecewise_invest.optimize(fx.solvers.HighsSolver())

In [ ]:
# Visualize the piecewise investment cost curve
# Access through the storage component after it's part of the FlowSystem
fs_piecewise_invest.components['ThermalStorage'].capacity_in_flow_hours.piecewise_effects_of_investment.plot(
    title='Storage Investment Costs (Economies of Scale)'
)

In [ ]:
# Compare: Simple vs Piecewise investment
print(
    f'Simple model:    {fs_simple_invest.solution["ThermalStorage|size"].item():.0f} kWh, {fs_simple_invest.solution["costs"].item():.2f} €'
)
print(
    f'Piecewise model: {fs_piecewise_invest.solution["ThermalStorage|size"].item():.0f} kWh, {fs_piecewise_invest.solution["costs"].item():.2f} €'
)

### Visualize Storage Operation and Energy Flows

In [ ]:
# Storage operation visualization
fs_piecewise_invest.statistics.plot.heatmap('ThermalStorage')

In [ ]:
fs_piecewise_invest.statistics.plot.sankey.flows()

## Key Concepts

### Time-Varying Conversion Factors

For temperature-dependent efficiency, use arrays with flow labels as keys:
```python
fx.LinearConverter(
    'HeatPump',
    inputs=[fx.Flow('Elec', bus='Electricity', size=150)],
    outputs=[fx.Flow('Heat', bus='Heat', size=500)],
    # Equation: Elec * COP = Heat * 1
    conversion_factors=[{'Elec': cop_array, 'Heat': 1}],
)
```

### Piecewise Linear Functions

For non-linear relationships (part-load efficiency), use `PiecewiseConversion`:
```python
fx.PiecewiseConversion({
    'Fuel': fx.Piecewise([
        fx.Piece(start=78, end=132),   # Segment 1
        fx.Piece(start=132, end=179),  # Segment 2
    ]),
    'Elec': fx.Piecewise([
        fx.Piece(start=25, end=50),    # Segment 1
        fx.Piece(start=50, end=75),    # Segment 2
    ]),
})
```

### When to Use Each Approach

| Approach | Use Case | Complexity |
|----------|----------|------------|
| Time-varying factors | Efficiency depends on external conditions (temp, price) | Low |
| PiecewiseConversion | Efficiency depends on load level | Medium |
| PiecewiseEffects | Non-linear costs (economies of scale) | Medium |

## Summary

You learned how to:

- Model **time-varying efficiency** using conversion factor arrays
- Use **PiecewiseConversion** for load-dependent efficiency curves
- Apply **PiecewiseEffects** for non-linear cost functions
- Choose the right approach for your modeling needs

### Next Steps

- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Multi-scenario planning
- **[08-large-scale-optimization](08-large-scale-optimization.ipynb)**: Computational efficiency techniques